In [1]:
gender = 2
income = 20000

In [2]:
import json
import pprint
import joblib
import pickle
import os
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [3]:
decoded_predictions = pd.DataFrame(columns = ["offer_id", "gender", "income", "reward", "channels", "difficulty", "duration", "offer_type"])
decoded_predictions
# Opening JSON file
f = open('offers_encoded.json')
# returns JSON object as a dictionary
data = json.load(f)
#open model
complete_knn = joblib.load('complete_offer.pkl')

C:\Users\kelly\Anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.24.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [4]:
decoded_predictions = pd.DataFrame(columns = ["offer_id", "gender", "income", "reward", "channels", "difficulty", "duration", "offer_type"])
decoded_predictions

,offer_id,gender,income,reward,channels,difficulty,duration,offer_type


In [5]:
# Opening JSON file
f = open('offers_encoded.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
def decode(dictionary):
    dictionary['offer_id'] = dictionary['offer_id'].map({8:'Offer 7: The Americano',
                             2:'Offer 2: The Cold Brew',
                            1:'Offer 8: The Espresso',
                            3:'Offer 6: The Pourover',
                            0:'Offer 9: The Macchiato',
                            5:'Offer 5: The French Press',
                            4:'Offer 4: The Mocha',
                            9:'Offer 3: The Latte',
                            7:'Offer 10: The Cappuccino',
                            6:'Offer 1: The Doppio'},
                             na_action=None)
    dictionary['gender'] = dictionary['gender'].map({0:'M',
                                                    1: 'O',
                                                    2: 'F'})
    dictionary['channels'] = dictionary['channels'].map({1: "email, mobile, social",
                                                         3: "web, email, mobile",
                                                         0: "web, email, mobile, social",
                                                         2: "web, email"})
    dictionary['offer_type'] = dictionary['offer_type'].map({1: "informational",
                                                            2: "bogo",
                                                            0: "discount"})
    dictionary['offer_completed_y_n'] = dictionary['offer_completed_y_n'].map({0:"Yes",
                                                                        1: "No"})
    global decoded_predictions
    decoded_predictions = decoded_predictions.append(dictionary)

g = gender
i = int(income)

def predictions():
    for dictionary in data:
        dictionary.update(gender = g, income = i,)
        complete_df = pd.DataFrame(dictionary, index=[0])[["offer_id", "gender", "income", "reward", "channels", "difficulty", "duration", "offer_type"]]
        complete_score = int(complete_knn.predict(complete_df))
        dictionary['offer_completed_y_n'] = complete_score
        new_dict = pd.DataFrame(dictionary, index=[0])
        decode(new_dict)

predictions()
f.close()
decoded_predictions.sort_values('offer_completed_y_n', ascending = False).drop(['gender', 'income'], axis=1)

,offer_id,reward,channels,difficulty,duration,offer_type,offer_completed_y_n
0,Offer 5: The French Press,0,"email, mobile, social",0,3,informational,Yes
0,Offer 7: The Americano,0,"web, email",0,4,informational,Yes
0,Offer 10: The Cappuccino,10,"email, mobile, social",10,7,discount,No
0,Offer 2: The Cold Brew,10,"web, email, mobile, social",10,5,discount,No
0,Offer 8: The Espresso,5,"web, email, mobile, social",5,5,discount,No
0,Offer 1: The Doppio,5,"web, email",5,7,discount,No
0,Offer 9: The Macchiato,3,"web, email, mobile, social",7,7,bogo,No
0,Offer 4: The Mocha,2,"web, email, mobile, social",10,10,bogo,No
0,Offer 3: The Latte,2,"web, email",10,7,bogo,No
0,Offer 6: The Pourover,5,"web, email, mobile",20,10,bogo,No
